In [1]:
#for local
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [ ]:
from datetime import datetime
import importlib
import numpy as np
import logging
import os
import shutil
from typing import Any
import yaml
import optuna
from rdkit import RDLogger
from generator import Generator
from language import Language
from node import MolSentenceNode
from utils import add_sep, class_from_package, make_logger
RDLogger.DisableLog('rdApp.*')
optuna.logging.disable_default_handler()

yaml_path = "config/optuna_generation.yaml"
with open(os.path.join(repo_root, yaml_path)) as f:
    conf = yaml.safe_load(f)
transition_args = conf.get("transition_args", {})
model_dir = os.path.join(repo_root, transition_args.pop("model_dir"))
lang_path = conf.get("lang_path")
if lang_path is None:
    lang_name = os.path.basename(os.path.normpath(model_dir)) + ".lang"
    lang_path = add_sep(model_dir) + lang_name
lang = Language.load(lang_path)
reward_class = class_from_package("reward", conf.get("reward_class"))
reward = reward_class(**conf.get("reward_args", {}))
filter_settings = conf.get("filters", [])
filters = []
for s in filter_settings:
    filter_class = class_from_package("filter", s.pop("filter_class"))
    filters.append(filter_class(**s))
generator_args = conf.get("generator_args", {})

def objective(trial):
    policy_class = trial.suggest_categorical("policy_class", ["UCB", "PUCT"])
    c = trial.suggest_loguniform("c", 0.01, 2)
    best_rate = trial.suggest_uniform("best_rate", 0, 1)
    rollout_width = trial.suggest_int("rollout_width", 1, 10)
    allow_rollout_overlaps = trial.suggest_categorical("allow_rollout_overlaps", [True, False])
    n_rollouts = trial.suggest_int("n_rollouts", 1, 10)
    n_tries = trial.suggest_int("n_tries", 1, 10)
    bottom_p = trial.suggest_loguniform("1-top_p", 0.002, 0.02)
    top_p = 1 - bottom_p
    temperature = trial.suggest_uniform("temperature", 0.8, 1.2)
    filtered_reward = trial.suggest_uniform("filtered_reward", -2, 0.2)

    output_dir=os.path.join(repo_root, "sandbox", conf["output_dir"], datetime.now().strftime("%m-%d_%H-%M")) + os.sep
    console_level = logging.ERROR
    file_level = logging.DEBUG if conf.get("debug") else logging.INFO
    logger = make_logger(output_dir, console_level=console_level, file_level=file_level)
    logger.info("params:" + str(trial.params))

    transition_class = class_from_package("transition", conf["transition_class"])
    transition_args["top_p"] = top_p
    transition_args["temperature"] = temperature
    transition = transition_class(model_dir=model_dir, lang=lang, logger=logger, device=conf.get("device"), **transition_args)
    
    policy_class = class_from_package("policy", policy_class)
    policy = policy_class(c=c, best_rate=best_rate)
    generator_args["policy"] = policy
        
    root = MolSentenceNode.bos_node(lang, device=conf.get("device")) # TODO: change after root node generalization
    
    generator_class = class_from_package("generator", conf.get("generator_class", "MCTS"))
    generator_args["filtered_reward"] = filtered_reward
    generator_args["rollout_width"] = rollout_width
    generator_args["allow_rollout_overlaps"] = allow_rollout_overlaps
    generator_args["n_rollouts"] = n_rollouts
    generator_args["n_tries"] = n_tries
    generator = generator_class(root=root, transition=transition, reward=reward, filters=filters, output_dir=output_dir, logger=logger, **generator_args)

    generator.generate(time_limit=conf.get("time_limit"), max_generations=conf.get("max_generations"))
    best_reward_rate = conf.get("best_reward_rate")
    mean_reward = generator.mean_reward(window=conf.get("mean_reward_window"))
    trial.set_user_attr("mean_reward", mean_reward)
    trial.set_user_attr("best_reward", generator.best_reward)
    return (1 - best_reward_rate) * mean_reward + best_reward_rate * generator.best_reward

def log_callback(study: optuna.Study, trial: optuna.Trial):
    val = trial.value
    print_trial(trial)
    
def print_trial(trial: optuna.Trial):
    print(f"Trial {trial.number} score={trial.value:.3f}, mean_reward={trial.user_attrs['mean_reward']:.3f}, best_reward={trial.user_attrs['best_reward']:.3f}, params={trial.params}")
    
def print_best_trials(study: optuna.Study):
    print("Optuna trials completed.")
    print("------ Best trials -----")
    best_trials = sorted([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE], key=lambda t: t.value, reverse=True)[:5]
    for t in best_trials:
        print_trial(t)

c:\Users\fsfsf\anaconda3\envs\v3-forge\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
c:\Users\fsfsf\anaconda3\envs\v3-forge\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 指定されたプロシージャが見つかりません。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
study = optuna.create_study(direction="maximize", study_name=conf.get("study_name"), storage="sqlite:///generation_result/optuna_d_score.db", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=conf.get("n_trials"), callbacks=[log_callback])
print_best_trials(study)

Trial 0 score=0.177, mean_reward=0.072, best_reward=0.421, params={'policy_class': 'UCB', 'c': 1.1805346343994227, 'best_rate': 0.3216831378235876, 'rollout_width': 3, 'allow_rollout_overlaps': True, 'n_rollouts': 2, 'n_tries': 6, '1-top_p': 0.008627714274871436, 'temperature': 1.1386610098478531, 'filtered_reward': -0.03002868874706288}
Trial 1 score=0.204, mean_reward=0.103, best_reward=0.441, params={'policy_class': 'UCB', 'c': 0.06586752586079238, 'best_rate': 0.6763655162087864, 'rollout_width': 8, 'allow_rollout_overlaps': False, 'n_rollouts': 10, 'n_tries': 1, '1-top_p': 0.008688041862370485, 'temperature': 0.9987216225241677, 'filtered_reward': -1.777585203280518}
Trial 2 score=0.235, mean_reward=0.169, best_reward=0.387, params={'policy_class': 'PUCT', 'c': 0.012240554470821606, 'best_rate': 0.6134962590747709, 'rollout_width': 10, 'allow_rollout_overlaps': True, 'n_rollouts': 6, 'n_tries': 1, '1-top_p': 0.01997965011739437, 'temperature': 1.032407605689511, 'filtered_reward':

In [ ]:
# continue
optuna.study.load_study(study_name="d_score", storage="sqlite:///generation_result/optuna_d_score.db")
study.optimize(objective, n_trials=3, callbacks=[log_callback])
print_best_trials(study)